In [2]:
import pandas as pd
import requests
import re
import os

from bs4 import BeautifulSoup

import selenium
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [33]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_links_process_and_find_translation(webpage_url):
    csv_file_names = []  # Store the names of the created CSV files

    try:
        # Send an HTTP GET request to the webpage_url
        response = requests.get(webpage_url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the webpage
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find the <meta> tag with property="mw:PageProp/toc"
            toc_meta_tag = soup.find('meta', attrs={'property': 'mw:PageProp/toc'})

            # Check if the <meta> tag exists
            if toc_meta_tag:
                # Find the <ul> (unordered list) tag that follows the <meta> tag
                ul_tag = toc_meta_tag.find_next('ul')

                # Find all the <a> (anchor) tags within the <ul> tag
                links = ul_tag.find_all('a')

                # Create a list of dictionaries, each containing a formatted link and its title
                formatted_links = []
                for link in links:
                    href = link.get('href')
                    formatted_link = f"https://en.wikipedia.org{href}"
                    title = link.get('title')
                    entry = {"Link": formatted_link, "Title": title}
                    formatted_links.append(entry)  # Append the dictionary to the list
                    print(f"Link: {formatted_link}, Title: {title}")

                # Process each link
                for link_entry in formatted_links:
                    languages = []
                    titles = []

                    # Get the English title from the dictionary
                    english_title = link_entry["Title"]

                    # Add an additional row for "Language" set to "en" and "Title" set to the English title
                    languages.append("en")
                    titles.append(english_title)

                    response = requests.get(link_entry["Link"])
                    soup = BeautifulSoup(response.content, 'html.parser')

                    # Find the interlanguage links
                    interlanguage_links = soup.find_all("li", class_="interlanguage-link")

                    # Extract and store the language and correct title of each interlanguage link
                    for interlanguage_link in interlanguage_links:
                        lang = interlanguage_link.find("a")['lang']
                        title = interlanguage_link.find("a")['title'].split(" – ")[0]
                        languages.append(lang)
                        titles.append(title)

                    # Create a pandas DataFrame for this link
                    data = {
                        "Language": languages,
                        "Title": titles
                    }
                    df = pd.DataFrame(data)

                    # Create a CSV file name based on the link title
                    csv_file_name = f"dataframes/{english_title}.csv"

                    # Save the DataFrame as a CSV file
                    df.to_csv(csv_file_name, index=False)

                    # Get the original title from the link
                    original_title = english_title

                    # Call add_translated_row_to_csv with csv_file_name and original_title as arguments
                    add_translated_row_to_csv(csv_file_name, original_title)

                    print(f"CSV saved: {csv_file_name}")

                    # Append the created CSV file name to the list
                    csv_file_names.append(csv_file_name)

            else:
                print("No <meta> tag with property='mw:PageProp/toc' found on the page.")

        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

    return csv_file_names  # Return the list of created CSV file names



In [ ]:

webpage_url = 'https://en.wikipedia.org/wiki/List_of_geometry_topics'
created_csv_files = scrape_links_process_and_find_translation(webpage_url)

In [28]:
def get_links(webpage_url):
    formatted_links = []  # Create an empty list to store the URLs
    
    try:
        # Send an HTTP GET request to the webpage_url
        response = requests.get(webpage_url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the webpage
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find the <meta> tag with property="mw:PageProp/toc"
            toc_meta_tag = soup.find('meta', attrs={'property': 'mw:PageProp/toc'})

            # Check if the <meta> tag exists
            if toc_meta_tag:
                # Find the <ul> (unordered list) tag that follows the <meta> tag
                ul_tag = toc_meta_tag.find_next('ul')

                # Find all the <a> (anchor) tags within the <ul> tag
                links = ul_tag.find_all('a')

                # Extract and append the href attribute of each <a> tag to the formatted_links list
                for link in links:
                    href = link.get('href')
                    formatted_link = f"https://en.wikipedia.org{href}"
                    formatted_links.append(formatted_link)  # Append the formatted URL to the list
                    title = link.get('title')
                    # Optionally, you can print the links if needed
                    # print(f"Link: {formatted_link}, Title: {title}")

            else:
                print("No <meta> tag with property='mw:PageProp/toc' found on the page.")

        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
    
    return formatted_links  # Return the list of formatted links


In [34]:
def two_level_scrape_links_process_and_find_translation(webpage_url):
    links = get_links(webpage_url)
    
    for link in links:
        scrape_links_process_and_find_translation(link)

webpage_url = 'https://en.wikipedia.org/wiki/Lists_of_mathematics_topics'
two_level_scrape_links_process_and_find_translation(webpage_url)


Link: https://en.wikipedia.org/wiki/Elementary_arithmetic, Title: Elementary arithmetic
Link: https://en.wikipedia.org/wiki/Decimal_arithmetic, Title: Decimal arithmetic
Link: https://en.wikipedia.org/wiki/Decimal_point, Title: Decimal point
Link: https://en.wikipedia.org/wiki/Numeral_system, Title: Numeral system
Link: https://en.wikipedia.org/wiki/Place_value, Title: Place value
Link: https://en.wikipedia.org/wiki/Face_value, Title: Face value
CSV saved: dataframes/Elementary arithmetic.csv
CSV saved: dataframes/Decimal arithmetic.csv
CSV saved: dataframes/Decimal point.csv
CSV saved: dataframes/Numeral system.csv
CSV saved: dataframes/Place value.csv
CSV saved: dataframes/Face value.csv
Link: https://en.wikipedia.org/wiki/Logic, Title: Logic
CSV saved: dataframes/Logic.csv
Link: https://en.wikipedia.org/wiki/Limit_(mathematics), Title: Limit (mathematics)
Link: https://en.wikipedia.org/wiki/Limit_of_a_function, Title: Limit of a function
Link: https://en.wikipedia.org/wiki/One-sided

In [36]:
def two_level_scrape_links_process_and_find_translation(webpage_url):
    links = get_links(webpage_url)
    
    for link in links:
        scrape_links_process_and_find_translation(link)

webpage_url = 'https://en.wikipedia.org/wiki/Pure_mathematics'
two_level_scrape_links_process_and_find_translation(webpage_url)

Link: https://en.wikipedia.org/wiki/Class_(set_theory), Title: Class (set theory)
Link: https://en.wikipedia.org/wiki/Morphism, Title: Morphism
Link: https://en.wikipedia.org/wiki/Map_(mathematics), Title: Map (mathematics)
Link: https://en.wikipedia.org/wiki/Binary_operation, Title: Binary operation
CSV saved: dataframes/Class (set theory).csv
CSV saved: dataframes/Morphism.csv
CSV saved: dataframes/Map (mathematics).csv
CSV saved: dataframes/Binary operation.csv


In [41]:
def two_level_scrape_links_process_and_find_translation(webpage_url):
    links = get_links(webpage_url)
    
    for link in links:
        scrape_links_process_and_find_translation(link)

webpage_url = 'https://en.wikipedia.org/wiki/Outline_of_algebra'
two_level_scrape_links_process_and_find_translation(webpage_url)

No <meta> tag with property='mw:PageProp/toc' found on the page.
Link: https://en.wikipedia.org#cite_note-26, Title: None
Link: https://en.wikipedia.org/wiki/Distributive_property, Title: Distributive property
CSV saved: dataframes/None.csv
CSV saved: dataframes/Distributive property.csv
Link: https://en.wikipedia.org#cite_note-NB3-32, Title: None
CSV saved: dataframes/None.csv
Link: https://en.wikipedia.org/wiki/Systems_of_linear_equations, Title: Systems of linear equations
Link: https://en.wikipedia.org/wiki/Linear_algebra, Title: Linear algebra
Link: https://en.wikipedia.org#cite_note-49, Title: None
CSV saved: dataframes/Systems of linear equations.csv
CSV saved: dataframes/Linear algebra.csv
CSV saved: dataframes/None.csv
Link: https://en.wikipedia.org/wiki/Complex_conjugate, Title: Complex conjugate
CSV saved: dataframes/Complex conjugate.csv
Link: https://en.wikipedia.org/wiki/Quantification_(logic), Title: Quantification (logic)
Link: https://en.wikipedia.org/wiki/Universal_qu

In [45]:
def two_level_scrape_links_process_and_find_translation(webpage_url):
    links = get_links(webpage_url)
    
    for link in links:
        scrape_links_process_and_find_translation(link)

webpage_url = 'https://en.wikipedia.org/wiki/List_of_Boolean_algebra_topics'
two_level_scrape_links_process_and_find_translation(webpage_url)

Link: https://en.wikipedia.org/wiki/Field_of_sets, Title: Field of sets
Link: https://en.wikipedia.org/wiki/User:Arthur_Rubin, Title: User:Arthur Rubin
Link: https://en.wikipedia.org/wiki/User_talk:Arthur_Rubin, Title: User talk:Arthur Rubin
Link: https://en.wikipedia.orgNone, Title: None
Link: https://en.wikipedia.org, Title: None
Link: https://en.wikipedia.org/wiki/User:Pmanderson, Title: User:Pmanderson
Link: https://en.wikipedia.orgNone, Title: None
Link: https://en.wikipedia.org, Title: None
CSV saved: dataframes/Field of sets.csv
CSV saved: dataframes/User:Arthur Rubin.csv
CSV saved: dataframes/User talk:Arthur Rubin.csv
Error: HTTPSConnectionPool(host='en.wikipedia.orgnone', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa1d5d12640>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Link: https://en.wikipedia.org/wiki/Two-element_Boolean_algebra, Title: Two

In [46]:
def two_level_scrape_links_process_and_find_translation(webpage_url):
    links = get_links(webpage_url)
    
    for link in links:
        scrape_links_process_and_find_translation(link)

webpage_url = 'https://en.wikipedia.org/wiki/Outline_of_algebraic_structures'
two_level_scrape_links_process_and_find_translation(webpage_url)

Link: https://en.wikipedia.org/wiki/Associativity, Title: Associativity
Link: https://en.wikipedia.org/wiki/Zero, Title: Zero
Link: https://en.wikipedia.org/wiki/Identity_element, Title: Identity element
Link: https://en.wikipedia.org/wiki/Inverse_element, Title: Inverse element
CSV saved: dataframes/Associativity.csv
CSV saved: dataframes/Zero.csv
CSV saved: dataframes/Identity element.csv
CSV saved: dataframes/Inverse element.csv
Link: https://en.wikipedia.org/wiki/Scalar_multiplication, Title: Scalar multiplication
Link: https://en.wikipedia.org#cite_note-2, Title: None
CSV saved: dataframes/Scalar multiplication.csv
CSV saved: dataframes/None.csv
Link: https://en.wikipedia.org/wiki/Associativity, Title: Associativity
Link: https://en.wikipedia.org/wiki/Commutativity, Title: Commutativity
Link: https://en.wikipedia.org/wiki/Additive_identity, Title: Additive identity
Link: https://en.wikipedia.org/wiki/Multiplicative_identity, Title: Multiplicative identity
Link: https://en.wikipedi

In [47]:
def two_level_scrape_links_process_and_find_translation(webpage_url):
    links = get_links(webpage_url)
    
    for link in links:
        scrape_links_process_and_find_translation(link)

webpage_url = 'https://en.wikipedia.org/wiki/List_of_abstract_algebra_topics'
two_level_scrape_links_process_and_find_translation(webpage_url)

Link: https://en.wikipedia.org/wiki/Set_(mathematics), Title: Set (mathematics)
CSV saved: dataframes/Set (mathematics).csv
Link: https://en.wikipedia.org/wiki/Subobject_classifier, Title: Subobject classifier
Link: https://en.wikipedia.org/wiki/Subquotient, Title: Subquotient
CSV saved: dataframes/Subobject classifier.csv
CSV saved: dataframes/Subquotient.csv
Link: https://en.wikipedia.org/wiki/If_and_only_if, Title: If and only if
Link: https://en.wikipedia.org/wiki/Closure_(mathematics), Title: Closure (mathematics)
Link: https://en.wikipedia.org#cite_note-FOOTNOTEKurzweilStellmacher19984-4, Title: None
Link: https://en.wikipedia.org#cite_note-FOOTNOTEKurzweilStellmacher19984-4, Title: None
CSV saved: dataframes/If and only if.csv
CSV saved: dataframes/Closure (mathematics).csv
CSV saved: dataframes/None.csv
CSV saved: dataframes/None.csv
Link: https://en.wikipedia.org/wiki/Field_(mathematics), Title: Field (mathematics)
Link: https://en.wikipedia.org/wiki/Vector_space, Title: Vecto

In [48]:
def two_level_scrape_links_process_and_find_translation(webpage_url):
    links = get_links(webpage_url)
    
    for link in links:
        scrape_links_process_and_find_translation(link)

webpage_url = 'https://en.wikipedia.org/wiki/List_of_abstract_algebra_topics'
two_level_scrape_links_process_and_find_translation(webpage_url)

Link: https://en.wikipedia.org/wiki/Set_(mathematics), Title: Set (mathematics)
CSV saved: dataframes/Set (mathematics).csv
Link: https://en.wikipedia.org/wiki/Subobject_classifier, Title: Subobject classifier
Link: https://en.wikipedia.org/wiki/Subquotient, Title: Subquotient
CSV saved: dataframes/Subobject classifier.csv
CSV saved: dataframes/Subquotient.csv
Link: https://en.wikipedia.org/wiki/If_and_only_if, Title: If and only if
Link: https://en.wikipedia.org/wiki/Closure_(mathematics), Title: Closure (mathematics)
Link: https://en.wikipedia.org#cite_note-FOOTNOTEKurzweilStellmacher19984-4, Title: None
Link: https://en.wikipedia.org#cite_note-FOOTNOTEKurzweilStellmacher19984-4, Title: None
CSV saved: dataframes/If and only if.csv
CSV saved: dataframes/Closure (mathematics).csv
CSV saved: dataframes/None.csv
CSV saved: dataframes/None.csv
Link: https://en.wikipedia.org/wiki/Field_(mathematics), Title: Field (mathematics)
Link: https://en.wikipedia.org/wiki/Vector_space, Title: Vecto

In [ ]:
def two_level_scrape_links_process_and_find_translation(webpage_url):
    links = get_links(webpage_url)
    
    for link in links:
        scrape_links_process_and_find_translation(link)

webpage_url = 'https://en.wikipedia.org/wiki/List_of_functional_analysis_topics'
two_level_scrape_links_process_and_find_translation(webpage_url)